In [26]:
pip install dnspython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 15.4 MB/s eta 0:00:00


Si por ejemplo queremos buscar cuales son los correos y números de teléfono que se encuentran en una página web podemos hacerlo de la siguiente manera. La página objetivo es la de contactos de la UAX ya que esta tiene más info y salen cosas mejores :)

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

original_url = "https://www.uax.com/contacto"

scraped = set()

emails = set()
phones = set()

try:
    response = requests.get(original_url)
except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
    pass

new_phones = set(re.findall(r"\+?3?4?\d?\d?\s\d\d\d?\s\d\d\d?\s\d\d\d?\s?\d?\d?\d?", response.text, re.I))
new_emails = set(re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", response.text))
emails.update(new_emails)
phones.update(new_phones)

soup = BeautifulSoup(response.text, "html.parser")
df_em = pd.DataFrame(emails, columns=["Email"])
df_ph = pd.DataFrame(phones, columns=["Phones"])
df_em.to_csv('email.csv', index=False)
df_ph.to_csv('phones.csv', index=False)

Vemos los Emails que hay

In [4]:
df_em

,Email
0,oficinadelestudiante@uax.es
1,inter@uax.es
2,admisiones@uax.es
3,fundacion@uax.es
4,info@UAX.es
5,info@uax.es
6,gaop@uax.es
7,videojs-youtube@3.0.0
8,musica@uax.es
9,psicoped@uax.es


Y los teléfonos que también hemos obtenido

In [6]:
df_ph

,Phones
0,91 375 73 73
1,91 810 92 00
2,+34 518 808 377
3,91 810 98 00
4,+34 91 910 17 67
5,91 810 93 00
6,91 440 23 30
7,91 810 99 90
8,91 810 97 00
9,91 810 90 00


Podemos ver ahora también todos las redirecciones que da la página que estamos observando. Con esto podemos obtener todas las redirecciones de una misma página para poder realizar también ciertos análisis en dichas páginas como el anterior de sacar números de teéfono y correos.
También se obtienen enlaces interesantes como cuentas de redes sociales y demás.

In [28]:
original_url = "https://www.uax.com/"
try:
    response = requests.get(original_url)
except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
    pass
soup = BeautifulSoup(response.text, "html.parser")
for attribute in soup.select("a"):
  try:
    aux = attribute.get("href").split("://")
    if len(aux) > 1:
      print(attribute.get("href"))
  except:
    pass

https://campus.uax.es/
https://www.uax.com/educacion
https://www.uax.com/campus-y-residencias
https://www.uax.com/centros-y-laboratorios
https://www.uax.com/instalaciones-deportivas
https://www.uax.com/campus-tour-interactivo
https://uax.moveonfr.com/publisher/1/eng
https://www.uax.com/online
https://wa.me/34518808377?text=
https://www.uax.com/online
https://wa.me/34518808377?text=
https://campus.uax.es/
https://www.uax.com/educacion
https://www.uax.com/campus-y-residencias
https://www.uax.com/centros-y-laboratorios
https://www.uax.com/instalaciones-deportivas
https://www.uax.com/campus-tour-interactivo
https://uax.moveonfr.com/publisher/1/eng
https://www.uax.com/noticias/tour-huawei-enterprise-roadshow-en-campus-uax-de-villanueva-de-la-canada
https://www.uax.com/campus-tour-interactivo
https://www.uax.com/oferta-academica?modalities%5B0%5D=719&program_types%5B0%5D=74&program_types%5B1%5D=75&program_types%5B2%5D=76
https://www.uax.com/presencial
https://www.uax.com/online
https://www.c

Podemos también jugar con las DNS y mirar datos que puedan ser de interes o relacionados.

In [29]:
import dns.resolver
import re

domain = 'uax.com'

response = dns.resolver.resolve(domain, 'SOA')

if response.rrset is not None:
    pattern= r'(%s)\.\s(\d{1,})\s(\w+)\sSOA\s(.*?)\.\s(.*?)\.\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})\s(\d{1,})' % domain
    match = re.match(pattern, str(response.rrset))
    m_name, ttl, class_, ns, email, serial, refresh, retry, expiry, minim = match.groups()

output ='''
Main Name In Zone: {a},
Cache TTL: {b},
Class: {c},
Authoritive NS: {d},
Email Address: {e},
Last Change: {f},
Retry In Secs: {g},
Expiry: {h},
Slave Cache In Sec: {i}
'''.format(a = m_name, b = ttl, c = class_, d = ns, e = str(email).replace('\\', ''), f = serial, g = retry, h = expiry, i = minim)

print(output)


Main Name In Zone: uax.com,
Cache TTL: 21600,
Class: IN,
Authoritive NS: zeus.uax.es,
Email Address: postmaster.uax.es,
Last Change: 2022060101,
Retry In Secs: 3600,
Expiry: 1209600,
Slave Cache In Sec: 86400



Como vemos podemos obtener el nombre del servidor como en este caso zeus.uax.es, su última fecha de modificacion y un correo.

A estás cosas anterirores se les pueden sumar varias cosas como encontrar también imágenes, archivos pdf, xls y demás pero son prácticamente iguales a las anteriores y para una introducción mas que suficiente ya que con código tampoco obtendremos los datos tan precisos y demás como al usar herramientas como maltego.